# sitq Interactive Tutorial

Welcome to the interactive tutorial for sitq! This notebook will guide you through the core concepts and features of the Simple Task Queue library.

## What is sitq?

sitq is a lightweight, async-first Python task queue library for running background jobs in small-to-medium services and tools.

### Key Features
- **Async-first API** with sync wrapper support
- **SQLite backend** for simple, file-based persistence
- **Cloudpickle serialization** for complex Python objects
- **Comprehensive error handling** with detailed context
- **Bounded concurrency** with configurable worker limits
- **Task scheduling** with ETA support
- **Result retrieval** with timeout handling

Let's get started!

## 1. Basic Setup

First, let's import sitq and set up a basic task queue.

In [ ]:
# Import sitq
import asyncio
from sitq import TaskQueue, Worker, SQLiteBackend

# Create a task queue with in-memory SQLite backend
# In production, you'd use a file-based backend like SQLiteBackend("tasks.db")
backend = SQLiteBackend(":memory:")
queue = TaskQueue(backend=backend)

print("✅ Task queue created successfully!")

## 2. Creating and Enqueuing Tasks

In sitq, you enqueue tasks by providing a function and its arguments. Let's create our first task!

In [ ]:
# Define a simple async function to be executed as a task
async def greet(name: str) -> str:
    """Simple greeting function."""
    await asyncio.sleep(0.1)  # Simulate some work
    return f"Hello, {name}! Welcome to sitq!"

# Define a sync function
def add_numbers(a: int, b: int) -> int:
    """Simple addition function."""
    return a + b

print("📝 Task functions defined")

In [ ]:
# Enqueue tasks (note: must be run in async context)
async def enqueue_tasks():
    # Enqueue async task
    task_id_1 = await queue.enqueue(greet, "Alice")
    print(f"🚀 Async task enqueued with ID: {task_id_1}")
    
    # Enqueue sync task
    task_id_2 = await queue.enqueue(add_numbers, 5, 3)
    print(f"🚀 Sync task enqueued with ID: {task_id_2}")
    
    return task_id_1, task_id_2

task_id_1, task_id_2 = await enqueue_tasks()

## 3. Processing Tasks with Workers

Workers execute tasks from the queue. Let's create a worker and process our tasks.

In [ ]:
# Create a worker
worker = Worker(backend)

print("👷 Worker created")

In [ ]:
# Process tasks with worker
async def process_tasks():
    print("⏳ Starting worker...")
    
    # Start worker in background
    worker_task = asyncio.create_task(worker.start())
    
    # Give worker time to process tasks
    await asyncio.sleep(1)
    
    # Stop worker
    await worker.stop()
    print("✅ Worker stopped")

await process_tasks()

In [ ]:
# Retrieve results
async def get_results():
    result_1 = await queue.get_result(task_id_1)
    if result_1 and result_1.status == "success":
        value_1 = queue.deserialize_result(result_1)
        print(f"Result 1: {value_1}")
    else:
        print(f"Result 1: Failed - {result_1.error if result_1 else 'Not found'}")
    
    result_2 = await queue.get_result(task_id_2)
    if result_2 and result_2.status == "success":
        value_2 = queue.deserialize_result(result_2)
        print(f"Result 2: {value_2}")
    else:
        print(f"Result 2: Failed - {result_2.error if result_2 else 'Not found'}")

await get_results()

## 4. Delayed Execution with ETA

You can schedule tasks for later execution using the `eta` parameter.

In [ ]:
from datetime import datetime, timezone, timedelta

async def delayed_task(message: str) -> str:
    """Task that executes after a delay."""
    return f"Executed at: {datetime.now(timezone.utc).isoformat()} | {message}"

# Schedule task for 1 second in the future
async def schedule_delayed_task():
    eta = datetime.now(timezone.utc) + timedelta(seconds=1)
    task_id = await queue.enqueue(delayed_task, "Delayed task", eta=eta)
    print(f"📅 Task scheduled for {eta.isoformat()}")
    
    # Start worker
    worker_task = asyncio.create_task(worker.start())
    await asyncio.sleep(2)
    await worker.stop()
    
    # Get result
    result = await queue.get_result(task_id)
    if result and result.status == "success":
        value = queue.deserialize_result(result)
        print(f"{value}")

await schedule_delayed_task()

## 5. Bounded Concurrency

Control the maximum number of tasks that can execute simultaneously using `max_concurrency`.

In [ ]:
import time

async def slow_task(task_id: int, sleep_time: float = 0.2) -> str:
    """Task that simulates work with a sleep."""
    start_time = time.time()
    await asyncio.sleep(sleep_time)
    end_time = time.time()
    return f"Task {task_id} completed in {end_time - start_time:.2f}s"

# Create worker with max_concurrency=2
bounded_worker = Worker(backend, max_concurrency=2)

async def test_concurrency():
    # Enqueue 5 tasks
    task_ids = []
    for i in range(1, 6):
        task_id = await queue.enqueue(slow_task, i, 0.2)
        task_ids.append(task_id)
    
    print(f"📦 Enqueued {len(task_ids)} tasks")
    print("Starting worker with max_concurrency=2...")
    
    # Start worker
    start_time = time.time()
    worker_task = asyncio.create_task(bounded_worker.start())
    
    # Wait for tasks to complete
    await asyncio.sleep(2)
    
    await bounded_worker.stop()
    end_time = time.time()
    
    print(f"⏱️ Total time: {end_time - start_time:.2f}s")
    
    # Retrieve results
    for i, task_id in enumerate(task_ids, 1):
        result = await queue.get_result(task_id)
        if result and result.status == "success":
            value = queue.deserialize_result(result)
            print(value)

await test_concurrency()

## 6. Error Handling

sitq captures task errors and provides detailed tracebacks.

In [ ]:
# Define a function that will fail
async def failing_task() -> float:
    """Task that raises a division by zero error."""
    return 1 / 0

async def test_error_handling():
    # Enqueue failing task
    task_id = await queue.enqueue(failing_task)
    print("🔴 Enqueued failing task")
    
    # Process with worker
    worker_task = asyncio.create_task(worker.start())
    await asyncio.sleep(1)
    await worker.stop()
    
    # Get result
    result = await queue.get_result(task_id)
    if result:
        print(f"Status: {result.status}")
        if result.status == "failed":
            print(f"Error: {result.error}")
            print(f"Traceback (last 3 lines):")
            traceback_lines = result.traceback.split("\n") if result.traceback else []
            for line in traceback_lines[-3:]:
                if line.strip():
                    print(f"  {line}")

await test_error_handling()

## 7. Best Practices

Based on what we've learned, here are some best practices for using sitq:

### ✅ Do's

1. **Always check result status** before deserializing
2. **Handle errors gracefully** with proper error checking
3. **Use appropriate concurrency limits** based on your workload
4. **Use timezone-aware datetimes** for ETA scheduling (UTC recommended)
5. **Stop workers cleanly** when done to ensure proper shutdown
6. **Use timeouts** when retrieving results to avoid hanging

### ❌ Don'ts

1. **Don't ignore task failures** - always check `result.status`
2. **Don't use blocking operations** in async tasks
3. **Don't forget to deserialize** results with `queue.deserialize_result()`
4. **Don't create too many workers** - respect system resources
5. **Don't use naive datetimes** for ETA - always make them timezone-aware

## 8. Next Steps

Congratulations! You've completed the sitq interactive tutorial. Here's what you can do next:

### 📚 Learn More
- [Quickstart](quickstart.md) - Get up and running in 5 minutes
- [Delayed Execution](delayed-execution.md) - Schedule tasks for later
- [Concurrency Control](concurrency.md) - Manage task parallelism
- [Failure Handling](failures.md) - Handle errors gracefully
- [API Reference](../reference/api/sitq.md) - Complete API documentation

### 🛠️ Try the Examples
Run the complete examples in `examples/basic/`:
- `01_end_to_end.py` - End-to-end workflow
- `02_eta_delayed_execution.py` - Delayed execution with ETA
- `03_bounded_concurrency.py` - Concurrency control
- `04_failures_and_tracebacks.py` - Error handling
- `05_sync_client_with_worker.py` - Sync wrapper usage

## 🎉 Thank You!

Thank you for trying out sitq! We hope this tutorial has been helpful.

**Happy task queuing! 🚀**